In [ ]:
%%html
<style>
h1, h2, h3, h4, h5 {
    color: darkblue;
    font-weight: bold !important;
}
h2 {
    border-bottom: 8px solid darkblue !important;
    padding-bottom: 8px;
}
h3 {
    border-bottom: 2px solid darkblue !important;
    padding-bottom: 6px;
}
.info, .success, .warning, .error {
    border: 1px solid;
    margin: 10px 0px;
    padding:15px 10px;
}
.info {
    color: #00529b;
    background-color: #bde5f8;
}
.success {
    color: #4f8a10;
    background-color: #dff2bf;
}
.warning {
    color: #9f6000;
    background-color: #FEEFB3;
}
.error {
    color: #D8000C;
    background-color: #FFBABA;
}
.language-bash {
    font-weight: 900;
}
.ex {
    font-weight: 900;
    color: rgba(27,27,255,0.87) !important;
}
.mn {
    font-family: Menlo, Consolas, "DejaVu Sans Mono", monospace
}
table {
    margin-left: 0 !important;}
</style>

# Day 3: Up and Running with Python

## 3.3 Netmiko

-   Most modern network devices support REST APIs but there are still many legacy network devices that could not be replaced easily.  Network engineers have to use CLI via SSH, telnet or serial interfaces to connect to those devices.  Netmiko is an open source library to automatate interface via CLI, telnet or serial[<sup>1</sup>](#fn1) to those devices.

-   Netmiko is a multi-vendor network automation library for Python.
-   It support SSH (via Paramiko library) and Telnet connection
-   It provides the following features:
    -   Establish SSH/Telnet connection to network device(s)
    -   Establish SSH connection via proxy SSH server
    -   Execution of show commands and retrieval of output data
    -   Execution of configuration commands
    
<span id="fn1">**1**:</span> Telnet and serial are not available in some more popular network devices.
    
### Installation

```bash
pip install netmiko
python -c "import netmiko; print(netmiko.__version__)"
```

In [ ]:
!python -c "import netmiko; print(netmiko.__version__)"

In [ ]:
import netmiko

netmiko.Netmiko??

In [ ]:
netmiko.BaseConnection??

### Documentation

-   There are many parameters and new ones may be added in the later release. We normally only need a few of them. Do read the docstring to find out more about the parameters.


-   **`device_type`** indicates the type of network device types that Netmiko is connected to.
    The list of supported device types are found in [`ssh_dispatcher.py`](https://github.com/ktbyers/netmiko/blob/e4669fa7b73660c1ecb2874ee2f66c71cf840691/netmiko/ssh_dispatcher.py).
    -   Common device types supporting ssh login include:
        -   `arista_eos`
        -   `cisco_saos`, `cisco_asa`, `cisco_ios`, `cisco_nxos`, `cisco_s300`, `cisco_tp`, `cisco_tp`, `cisco_wlc`, `cisco_xe`, `cisco_xr`
        -   `f5_ltm`, `f5_tmsh`, `f5_linux`
        -   `fortinet`
        -   `hp_comware`, `hp_procurve`
        -   `huawei`, `huawei_vrpv8`,
        -   `juniper`, `juniper_junos`
    -   Common device types supporting file transfer include:
        -   `arista_eos`
        -   `cisco_asa`, `cisco_ios`, `cisco_nxos`, `cisco_xe`, `cisco_xr`
        -   `juniper_junos`
        -   `linux`
    -   Common device types supporting telnet include:
        -   `apresia_aeos_telnet`
        -   `arista_eos_telnet`
        -   `cisco_ios_telnet`
        -   `hp_procurve_telnet`, `hp_comware_telnet`
        -   `juniper_junos_telnet`
        -   `huawei_telnet`
    -   Common device types supporting serial include:
        -   `cisco_ios_serial`
    -   Common device types supporting ssh terminal server include:
        -   `terminal_server`
        -   `autodetect`

<span class="ex">Example: <span class='mn'>send_command()</span></span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

device = Netmiko(
    #ip=input('Enter IP address: '),  # 192.168.99.3
    ip='192.168.88.3',
    username='admin',
    #password=getpass('Password: '),  # cisco
    password='cisco',
    device_type='cisco_ios',
    verbose=True,
    secret='cisco'
)

output = device.send_command("show ip int brief")
print(output)

**Available commands from device**

In [ ]:
print(dir(device))

### Unexpected Command Prompt

The above command works because the command prompt after execution of the command is expected.

However, when the command asks for input or generate other non-expected text, the missing "expected-prompt" will cause the function `send_command()` fails.

For example, if we executes the following command via `send_command()`:

<pre>
<span class='ex'>copy flash:/c880data-universalk9-mz.154-2.T1.bin flash:/test1.bin</span>
</pre>

The output of this command looks as follows (on the router's CLI):

<pre>
<span class='ex'>
# copy flash:/c880data-universalk9-mz.154-2.T1.bin flash:/test1.bin
Destination filename [test1.bin]? 
Copy in progress...CCCCCCCCCCCCCCCCCCCC
...   [large number of C's omitted]
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
42628912 bytes copied in 143.676 secs (296702 bytes/sec)
</span>
</pre>

In this case, we could use the `'expect_string'` argument to `send_command()`, which tells Netmiko what to look for in the output.

<span class='ex'>Example: Use <span class='mn'>expect_string</span> to end expected command completion</span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

device = Netmiko(
    #ip=input('Enter IP address: '),  # 192.168.99.2
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    verbose=True,
    secret='cisco'
)
device.enable()

cmd = 'copy flash:/c880data-universalk9-mz.154-2.T1.bin flash:/test1.bin'

output = device.send_command(
    cmd,
    expect_string=r'Destination filename' 
    # Remote expects input from users via this prompt
)
try:
    output += device.send_command('\n', expect_string=r'#')
except:
    pass
finally:
    device.disconnect()
    print(output)

### Longer Command Response Time

-   Some commands may take longer to return. We could use `delay_factor` to let Netmiko to wait longer.


-   `delay_factor` is an argument to `send_command()`, which multiple existing timeout.

<span class='ex'>Example: Use <span class='mn'>delay_factor</span> to increase command response time</span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

device = Netmiko(
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    verbose=False,
    secret='class'
)

output = device.send_command(
    "show ip int brief",
    delay_factor=2
)
device.disconnect()
print(output)

-   We could also use `global_delay_factor` which multiple **ALL** timeout.

<span class='ex'>Example: Use <span class='mn'>global_delay_factor</span> to increase all response time</span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

device = Netmiko(
    #ip=input('Enter IP address: '),  # 192.168.99.2
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    verbose=False,
    secret='cisco',
    global_delay_factor=2
)
device.enable()

output = device.send_command("show run")
device.disconnect()
print(output)

<span class='ex'>Example: Save command output to file</span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

outputfile = 'cisco.log'

device = Netmiko(
    #ip=input('Enter IP address: '),  # 192.168.99.2
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    secret='cisco',                  # privilege password
    verbose=False,
    global_delay_factor=1,
)

cmds = [
    'show run',
    'show int',
    'show ip int'
]

with open(outputfile, 'w') as f:
    for cmd in cmds:
        f.write(device.send_command(cmd))

device.disconnect()

<span class='ex'>Example: Use <span class='mn'>send_config_set()</span> to send configuration commands</span>

In [ ]:
from netmiko import Netmiko
from getpass import getpass

outputfile = 'cisco.log'

device = Netmiko(
    #ip=input('Enter IP address: '),  # 192.168.99.2
    ip='192.168.88.3',
    username='admin',
    password='cisco',
    device_type='cisco_ios',
    secret='cisco',                  # privilege password
    verbose=True,
    global_delay_factor=1,
)
device.enable()

cfg_commands = [
    'no logging console',              # Disable console logging
    'logging buffered 64000',          # Configure 64 kb logging buffer
    'logging buffered informational',  # Access-list violation logging    
]

output = device.send_config_set(cfg_commands)

device.disconnect()

with open(outputfile, 'w') as f:
    f.write(output)

<span class='ex'>Example: Use <span class='mn'>send_config_from_file()</span> to send configuration commands from file</span>

In [ ]:
%%file ./console_logging.txt
logging buffered 8000
logging console

In [ ]:
from netmiko import Netmiko
from getpass import getpass

output = 'cisco.log'
config = 'console_logging.txt'

devcfg = {
    'ip'                 : input('Enter IP address: '),  # 192.168.99.3
    'username'           : 'admin',
    'password'           : getpass(),
    'device_type'        : 'cisco_ios',
    'secret'             : 'class',
    'verbose'            : False,
    'global_delay_factor': 1
}

device = Netmiko(**devcfg)
device.enable()

cfg_commands = {
    'no logging console',              # Disable console logging
    'logging buffered 64000',          # Configure 64 kb logging buffer
    'logging buffered informational',  # Access-list violation logging    
}

output = device.send_config_from_file(config)

device.disconnect()

with open(outputfile, 'w') as f:
    f.write(output)

## netmiko_tools

-   The author of netmiko has written [3 scripts](
https://github.com/ktbyers/netmiko_tools/tree/master/netmiko_tools) to faclitate common network automation commands, without writing any Python code:

    1.  `netmiko-show`
    1.  `netmiko-grep`
    1.  `netmiko-cfg`

-   These scripts uses GNU grep.  You may download a Windows binary from [here](http://gnuwin32.sourceforge.net/packages/grep.htm).  *You may want to use a virus scanner to scan for virus or trojan house in these binaries. I normally use [Virus Total].(https://www.virustotal.com/#/home/upload)*


-   You may modify these scripts to refer to a local copy of `./grep.exe`

```Bash
GREP = '/bin/grep'
if os.name == 'nt':
    GREP = r'C:\MyLesson\NAWP\netmiko_tools\grep.exe'
elif not os.path.exists(GREP):
    GREP = '/usr/bin/grep'
NETMIKO_BASE_DIR = '~/.netmiko'
ERROR_PATTERN = "%%%failed%%%"
__version__ = '0.1.0'
```


-   A working copy of grep.exe, scripts may be available inside `.\netmiko_tools`.


-   These scripts will look for `.netmiko.yml` in the current directory or user's home directory for network devices.


-   A sample of `.netmiko.yml` is shown below.

In [ ]:
%%file .netmiko.yml
---
# Device connection information
web01:
    ip         : 192.168.99.2
    username   : admin
    password   : class
    device_type: cisco_ios
    secret     : class

web02:
    ip         : 192.168.99.3
    username   : admin
    password   : class
    device_type: cisco_ios
    secret     : class

web03:
    ip         : 192.168.99.4
    username   : admin
    password   : class
    device_type: cisco_ios
    secret     : class
        
# Lists of groups of devices:
cisco:
    - web01
    - web02

san:
    - web02
    - web03
    
all:
    - web01
    - web02
    - web03

<span class='ex'>Example: <span class='mn'>netmiko-show.py</span></span>

In [ ]:
!python ./netmiko_tools/netmiko-show.py --cmd "sh ver" cisco

In [ ]:
!python ./netmiko_tools/netmiko-show.py --cmd "sh ver" web02

In [ ]:
!python ./netmiko_tools/netmiko-grep.py --cmd "sh ip int" "administratively down" web02

In [ ]:
!python ./netmiko_tools/netmiko-cfg.py --cmd "logging buffered 64000" web02